# Reconstruct hologram in off-axis configuration

In [1]:
%matplotlib notebook
from time import sleep
from pymba import Vimba, VimbaException
from typing import Optional
import cv2
from pymba import Frame
import numpy as np
import NanoImagingPack as nip
import time
import threading
import matplotlib.pyplot as plt

from ipywidgets import *


Problem setting classpath. Switching to conventional viewer by setting __DEFAULTS__['IMG_VIEWER'] = 'NIP_VIEW' 


In [2]:
class VimbaHolo:
    def __init__(self):
        self.vimba = Vimba()
        self.vimba.startup()
        self.camera = self.vimba.camera(0)
        self.camera.open()
        self.is_streaming = False
        self.camera.arm('SingleFrame')
        self.Nx=self.Ny=400
        self.windowid = str(np.squeeze(np.random.randint(10000)))
        
        
        self.frame = None
        

        self.comp_phase_1 = 5e-4
        self.comp_phase_2 = -2.5e-4
        self.cy, self.cx = 201,1045


    
    def close(self):
        self.stopstreaming()
        self.camera.disarm()
        self.camera.close()
        self.vimba.shutdown()
    
    def set_exposuretime(self, value=50000):
        # read a feature value
        FEATURE_NAME = "ExposureTime"
        feature = self.camera.feature(FEATURE_NAME)
        feature.value = value
        print("Setting Value to: "+str(value))
        
    def get_exposuretime(self):
        FEATURE_NAME = "ExposureTime"
        feature = self.camera.feature(FEATURE_NAME)
        return feature.value
        
        
    def framegenerator(self, callback=None):
        while(self.is_streaming):
            self.frame = self.camera.acquire_frame().buffer_data_numpy()
            if callback is not None:
                callback(self.frame)
        
    def startstreaming(self, callback=None):
        if not self.is_streaming:
            self.is_streaming = True
            self.task = threading.Thread(target=self.framegenerator, args=(callback,))
            self.task.start()

        
    def stopstreaming(self):
        if self.is_streaming:
            self.is_streaming = False
            self.task.join()
        
    
    def get_ft(self):
        return nip.ft(self.frame)
    
    def set_cx(self, cx):
        self.cx = cx
    
    def set_cy(self, cy):
        self.cy = cy
        
    def set_comp_phase_1(self, value=0):
        self.comp_phase_1 = value
     
    def set_comp_phase_2(self, value=0):
        self.comp_phase_2 = value
        
    def AbsNormInt8(self, image, is_ft = False):
        if is_ft:
            image = np.log(1+np.abs(image))
        image /= np.max(image)
        image *=255
        return image
        
    def displayer(self, frame):
        # compute the fourier transform and add a parabolic phase term too shift back the reference
        comp_phase = np.exp(1j*nip.rr2(frame.shape)*self.comp_phase_1)
        frame_ft_comp = nip.ft(frame*comp_phase)
        frame_ft_comp_abs = self.AbsNormInt8(frame_ft_comp, is_ft=True)

        # extract CC term
        frame_ft_cc = nip.extract(frame_ft_comp, (self.Nx, self.Ny), (self.cx, self.cy), checkComplex=False)
        
        # compensate phase in reconstruction
        comp_phase_2 = np.exp(1j*nip.rr2(frame_ft_cc.shape)*self.comp_phase_2)
        # reconstruct 
        frame_amp = nip.ift(frame_ft_cc)*comp_phase_2
        frame_ang = self.AbsNormInt8(np.angle(frame_amp))
        frame_abs = self.AbsNormInt8(np.abs(frame_amp))

        # display stuff
        cc_term_norm = self.AbsNormInt8(frame_ft_cc, is_ft=True)
        

        image_1 = np.hstack((frame_ft_comp_abs, frame))
        image_2 = np.hstack((frame_abs,frame_ang, cc_term_norm))

        image_1 = cv2.resize(image_1, dsize = None, fx=.4, fy=.4)

        # display image
        cv2.imshow('Image' + self.windowid, np.uint8(image_1))
        cv2.imshow('Image FT' + self.windowid, np.uint8(image_2))
        if cv2.waitKey(1) == 27: 
            return  # esc to quit



        
        

In [3]:
camera = VimbaHolo()


In [4]:
callbackfct = camera.displayer
camera.startstreaming(callbackfct)

In [5]:
@widgets.interact(exposuretime=(0, 100))
def exposuretime(exposuretime=5):
    camera.set_exposuretime(exposuretime*1000)

@widgets.interact(cx=(0, 1000))     
def cx(cx=camera.cx):
    camera.set_cx(cx)

@widgets.interact(cy=(0, 1500))     
def cy(cy=camera.cy):
    camera.set_cy(cy)
    
@widgets.interact(compphase1=(0, 7.))   
def compphase1(compphase1=camera.comp_phase_1):
    camera.set_comp_phase_1(compphase1*1e-4)

@widgets.interact(compphase2=(-5., 5.))     
def compphase2(compphase2=camera.comp_phase_2):
    camera.set_comp_phase_2(compphase2*1e-4)

interactive(children=(IntSlider(value=5, description='exposuretime'), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=1000, description='cx', max=1000), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=201, description='cy', max=1500), Output()), _dom_classes=('widget-inter…

interactive(children=(FloatSlider(value=0.0005, description='compphase1', max=7.0), Output()), _dom_classes=('…

interactive(children=(FloatSlider(value=-0.00025, description='compphase2', max=5.0, min=-5.0), Output()), _do…

In [9]:

camera.stopstreaming()

In [10]:
camera.close()

In [7]:
camera.Nx=camera.Ny=300